# 6. Chroma DB と AWS Bedrock を使って、より高度な質問に対応する

このノートブックでは、Langchain, Chroma DB, AWS Bedrock の設定を行います。

- **AWS Bedrock の設定:** AWS の Bedrock のLLM が自然言語の理解と計算を担当します

- **Chroma DB Client の初期化:** 次に、ベクターDBである Chroma DB との接続を確立します。

- **Langchain の設定:** Langchainを設定します。Langchainは、言語モデルを外部のDBとシームレスに接続します。

- **実践的なサンプルを実行:** 設定が完了したら、関数を実行します。この関数は、Langchain, Bedrock, Chroma DB をどのように組み合わせて質問に答えるかを実演しています。Chroma DB に格納されたベクトルデータを使い、言語モデルに関連する情報を与え、回答の正確性と関連度を強化しています。

### 6.1 AWS Bedrock の設定

まずは AWS Bedrock の設定から始めましょう。
以下の関数では、あらかじめ指定された環境変数を Langchain の **default** として利用しています。

In [34]:
import os

def create_aws_credentials_file():
    # Retrieve environment variables
    aws_region = os.environ.get('AWS_DEFAULT_REGION', 'default_region')
    aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID', 'default_access_key')
    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY', 'default_secret_key')

    # Define the folder and file paths
    aws_folder_path = os.path.expanduser('~/.aws')
    credentials_file_path = os.path.join(aws_folder_path, 'credentials')

    # Create the .aws directory if it does not exist
    if not os.path.exists(aws_folder_path):
        os.makedirs(aws_folder_path)

    # Write the credentials to the file
    with open(credentials_file_path, 'w') as credentials_file:
        credentials_file.write('[default]\n')
        credentials_file.write(f'aws_access_key_id={aws_access_key_id}\n')
        credentials_file.write(f'aws_secret_access_key={aws_secret_access_key}\n')
        credentials_file.write(f'region={aws_region}\n')

    print(f"AWS の Credential を、以下のディレクトリに作成しました: {credentials_file_path}")
    
create_aws_credentials_file()    

AWS の Credential を、以下のディレクトリに作成しました: /home/cdsw/.aws/credentials


### 6.2 Chroma の初期設定

次に、Chroma DB を初期設定します。
ベクターDBにデータを入れるためには、**5_populate_local_chroma_db/populate_chroma_vectors.py** を利用してください。

コレクション名は **cml-default** とします。

In [35]:
import chromadb

persistent_client = chromadb.PersistentClient(path="/home/cdsw/chroma-data")
COLLECTION_NAME = "cml-default"

collection = persistent_client.get_collection(COLLECTION_NAME)

In [36]:
# レコード数の確認
record_count = collection.count()
print(f"レコード数: {record_count}")

レコード数: 15


Chroma DB にベクトルを格納するためのエンべディングモデルは、 [sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) を利用します。

In [37]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"

EMBEDDING_FUNCTION = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME)

In [38]:
query_embeddings = EMBEDDING_FUNCTION.embed_query("Icebergって何ですか？")

In [41]:
# query_embeddings の中身は 768次元のベクトル
len(query_embeddings)

768

In [42]:
# 特定の条件を持つクエリ
query_results = collection.query(
    query_embeddings=query_embeddings,
    n_results=2)

In [43]:
query_results

{'ids': [['/home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/requirements-azure/topics/ml-requirements-azure.txt',
   '/home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/architecture-overview/topics/ml-architecture-overview-provisioning.txt']],
 'distances': [[1.9136815209159268, 1.9523082961054916]],
 'metadatas': [[{'classification': 'public'}, {'classification': 'public'}]],
 'embeddings': None,
 'documents': [['Azure Account Requirements forCloudera Docs Azure Account Requirements for  The requirements for using  in Azure are described in Working with Azure     environments, which is linked in the Related information section, below. In addition, Cloudera Machine Learning on Azure has a few additional requirements:  Cloudera Machine Learning requires one Azure virtual network. Each Cloudera AI Workbench requires its own subnet. Each Cloudera AI Workbench requires its own NFS file share. Azure Files NFS 4.1, or Azure NetApp Files, are the recommended servic

### 6.3 Langchain の利用を開始する

Langchain を設定しましょう。まずは先ほど persistent_client に設定した Chroma のクライアントを、ベクトルストアとして設定します。

In [44]:
from langchain.vectorstores import Chroma

# Chroma のコレクション名を指定し、先ほど作成した persistent_client をベクトルストアの client として指定する
vectorstore = Chroma(
        client=persistent_client,
        collection_name=COLLECTION_NAME,
        embedding_function=EMBEDDING_FUNCTION
    )

AWS Bedrock は、AWSのクレデンシャルとモデル名を利用して Langchain とともに使用することができます。
今回は例として、 **anthropic.claude-v2:1** をモデルとして利用します。

In [45]:
from langchain.llms import Bedrock

LLM_MODEL = Bedrock(
    credentials_profile_name="default", model_id="anthropic.claude-v2:1"
)

シンプルなプロンプトを使って、LLMに文脈を使って回答するように指示してみましょう。
質問と回答のチェーンの中で、プロンプトのテンプレートを使います。

In [46]:
from langchain.prompts import PromptTemplate

# Prompt Template for Langchain
template = """あなたは有用なAIアシスタントです。以下の与えられたcontextだけを使って、以下の質問に回答してください。知らない場合は「存じ上げません」と回答してください。
文脈: {context}

質問: {question}

回答:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

最後に、これまでに設定した Bedrock のモデル、Chroma のベクトルストア、プロンプトテンプレートのすべてを利用して、文脈を考慮した質問回答のやりとりをしてみましょう。

In [47]:
from langchain.chains import RetrievalQA

# Chroma をベクトルストアとして利用したQAチェーンを作成
qa_chain = RetrievalQA.from_chain_type(
        llm=LLM_MODEL,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

ここまで来れば、ドキュメントの内容を考慮したQAを行うことができます。

**QUESTION　の変数の値を、任意の質問に変えてみましょう。**

In [55]:
# QA チェーンを実行し文脈を取得
QUESTION = "ML Runtimeって何ですか？"

embedded_question = EMBEDDING_FUNCTION.embed_query(QUESTION)

vector_query_results = collection.query(
    query_embeddings=query_embeddings,
    n_results=1)

context = vector_query_results['documents'][0][0]

In [49]:
vector_query_results

{'ids': [['/home/cdsw/data/https:/docs.cloudera.com/machine-learning/cloud/requirements-azure/topics/ml-requirements-azure.txt']],
 'distances': [[1.9136815209159268]],
 'metadatas': [[{'classification': 'public'}]],
 'embeddings': None,
 'documents': [['Azure Account Requirements forCloudera Docs Azure Account Requirements for  The requirements for using  in Azure are described in Working with Azure     environments, which is linked in the Related information section, below. In addition, Cloudera Machine Learning on Azure has a few additional requirements:  Cloudera Machine Learning requires one Azure virtual network. Each Cloudera AI Workbench requires its own subnet. Each Cloudera AI Workbench requires its own NFS file share. Azure Files NFS 4.1, or Azure NetApp Files, are the recommended services, in order of           preference. For Azure Files NFS 4.1, only NFS version 4.1 is supported. For Azure NetApp           Files, only NFS version 3 is supported. We require the No Root Squ

In [51]:
# 文脈を考慮してQAチェーンを call するための関数
def generate_response_with_context(question, context, qa_chain):
    result = qa_chain({"query": question, "context": context})
    return result["result"]

print(f"質問: {QUESTION} \n \n回答:{generate_response_with_context(QUESTION, context, qa_chain)}") 

質問: ML Runtimeって何ですか？ 
 
回答: ML Runtimesとは、Cloudera Machine Learningの中核をなすコンポーネントの1つで、データサイエンスのワークロードを実行し、基盤となるクラスタへのアクセスを仲介する役割を担っています。

具体的には、ML RuntimesはKubernetes上でコンテナとして実行され、Python、R、Scalaなどのワークロードの実行環境を提供します。データサイエンティストは、このML Runtimesコンテナの中でインタラクティブなセッションを開始したり、ジョブをスケジュールしたり、モデルをデプロイしたりすることができます。

ML Runtimesには、使用事例に応じて目的別に最適化されたイメージが用意されており、単一のエディタや言語カーネル、ツールやライブラリのセットが予めインストールされています。これにより、データサイエンティストは求める実行環境を素早く利用できるようになっています。


### 6.4 完成版

In [53]:
import langchain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.llms import Bedrock
import chromadb

In [54]:
# Clientの設定
persistent_client = chromadb.PersistentClient(path="/home/cdsw/chroma-data")
COLLECTION_NAME = "cml-default"

# コレクションの取得
collection = persistent_client.get_or_create_collection(COLLECTION_NAME)

In [56]:
EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"
EMBEDDING_FUNCTION = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# Chroma DB の設定
vectorstore = Chroma(
        client=persistent_client,
        collection_name=COLLECTION_NAME,
        embedding_function=EMBEDDING_FUNCTION
    )

LLM_MODEL = Bedrock(
    credentials_profile_name="default", model_id="anthropic.claude-v2:1"
)

In [57]:
# Langchain のプロンプトのテンプレート
template = """あなたは有用なAIアシスタントです。以下の与えられたcontextだけを使って、以下の質問に回答してください。知らない場合は「存じ上げません」と回答してください。
Context:{context}
>>質問<<{question}
>>回答<<"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


# Chroma のベクトルストアを使って QA チェーンを生成
qa_chain = RetrievalQA.from_chain_type(
        llm=LLM_MODEL,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

def generate_response_with_context(question, context, qa_chain):
    result = qa_chain({"query": question, "context": context})
    return result["result"]

# QA チェーンの実行と文脈の取得
QUESTION = "ML Runtime って何ですか？"

embedded_question = EMBEDDING_FUNCTION.embed_query(QUESTION)

vector_query_results = collection.query(
    query_embeddings=query_embeddings,
    n_results=1)

context = vector_query_results['documents'][0][0]

print(f"質問: {QUESTION} \n \n回答:{generate_response_with_context(QUESTION, context, qa_chain)}") 

質問: ML Runtime って何ですか？ 
 
回答: Cloudera Machine Learningのアーキテクチャ概要によると、Cloudera AI Runtimesはデータサイエンスワークロードを実行し、基盤となるクラスターへのアクセスを仲介する責任があります。

Cloudera AI Runtimesは、特定のユースケースに合わせて目的を持って構築されています。単一のエディター(例: Workbench、Jupyterlab)で利用でき、単一の言語カーネル(例: Python 3.8 または R 4.0)を搭載しており、一連のUNIXツールとユーティリティまたは言語ライブラリとパッケージを備えています。

つまり、Cloudera AI Runtimesはデータサイエンスワークロードを実行するためのコンテナイメージのことを指します。これにより、データサイエンティストはPython、R、Scalaなどのワークロードを分離された実行環境で実行できます。


### 6.5 要約

このノートブックでは、Langchain, Chroma DB, AWS Bedrock を活用して、文脈（＝固有ドメインの知識）を考慮した質問応答の流れを実演しました。
LLMとデータ検索システムの統合に関して理解を深め、活用へのイメージを持つことができたでしょうか。

### 次のステップ: ディレクトリ7を見てみましょう